# Processing data with pandas

```{attention}
Follow the lesson and fill in your notebooks using Binder.<br/>
<a href="https://mybinder.org/v2/gh/jdaled/geo-python_UPD/dev-upd?urlpath=lab/tree/source/notebooks/L5/processing-data-with-pandas_eq.ipynb"><img alt="Binder badge" src="https://img.shields.io/badge/launch-binder-red.svg" style="vertical-align:text-bottom"></a>
```

During the first part of this lesson you learned the basics of pandas data structures (*Series* and *DataFrame*) and got familiar with basic methods loading and exploring data.
Here, we will continue with basic data manipulation and analysis methods such calculations and selections.

We are now working in a new notebook file and we need to import pandas again. 

In [ ]:
import pandas as pd

Let's work with the same input data `'usgs_eqdata_2019-01.csv'` and load it using the `pd.read_csv()` method. Remember, that the first 8 lines contain metadata so we can skip those. This time, let's store the filepath into a separate variable in order to make the code more readable and easier to change afterwards: 

In [ ]:
# Define file path:
fp = "usgs_eqdata_2019-01.csv"

# Read in the data from the file (starting at row 4):
data = pd.read_csv(fp, skiprows=3)
data["time"] = pd.to_datetime(data["time"], infer_datetime_format=True) # convert time to datetime

Remember to always check the data after reading it in:

In [ ]:
data.head()

## Basic operations

One of the most common things to do in pandas is to create new columns based on calculations between different variables (columns).

We can create a new column `DIFF` in our DataFrame by specifying the name of the column and giving it some default value (in this case the decimal number `0.0`).

In [ ]:
# Define a new column "DIFF"
data["DIFF"] = 0.0

# Check how the dataframe looks like:
data.head()

Let's check the datatype of our new column:

In [ ]:
data["DIFF"].dtypes

Okey, so we see that pandas created a new column and recognized automatically that the data type is float as we passed a 0.0 value to it.

Let's update the column `DIFF` by calculating the difference between `latitude` and `longitude` columns to get an idea how much the temperatures have been varying during different days:

In [ ]:
# Calculate difference of the values of latitude and longtiude,
# just for the sake of an example...
data["DIFF"] = data["latitude"] - data["longitude"]

# Check the result
data.head()

The calculations were stored into the `DIFF` column as planned. 

You can also create new columns on-the-fly at the same time when doing the calculation (the column does not have to exist before). Furthermore, it is possible to use any kind of math
algebra (e.g. subtraction, addition, multiplication, division, exponentiation, etc.) when creating new columns.

We can for example convert the magnitudes in the `mag` column into seismoic using the formula $M_w=exp(0.741+0.210m_b)−0.785$ (equation 13 in [Lolli et al., 2014](https://academic.oup.com/gji/article/199/2/805/616348)). Let's do that and store it in a new column called `Mw` [moment magnitude](https://www.usgs.gov/programs/earthquake-hazards/magnitude-types).

In [ ]:
import numpy as np

# just loading the data again because I don't like the DIFF column...
data = pd.read_csv(fp, skiprows=3)
data["time"] = pd.to_datetime(data["time"], infer_datetime_format=True) # convert time to datetime

# Create a new column and convert mb (body wave magnitude) to Mw (Moment magnitude)
data["Mw"] = 0.0
data["Mw"] = np.exp(0.741 + (0.210*data["mag"])) - 0.785

# Check output
data.head()

#### Check your understanding

Calculate the difference between the body-wave magnitude and moment magnitude **and store the result a new column** calle `mDiff` in our dataframe.

In [ ]:
# Solution

data['mDiff'] = data['Mw'] - data['mag']
data

## Selecting rows and columns

We often want to select only specific rows from a DataFrame for further analysis. There are multiple ways of selecting subsets of a pandas DataFrame. In this section we will go through the most useful tricks for selecting specific rows, columns and individual values.

### Selecting several rows

One common way of selecting only specific rows from your DataFrame is done via **index slicing** to extract part of the DataFrame. Slicing in pandas can be done in a similar manner as with normal Python lists, i.e. you specify the index range you want to select inside the square brackets: ``dataframe[start_index:stop_index]``.

Let's select the first five rows and assign them to a variable called `selection`:

In [ ]:
# Select first five rows of dataframe using row index values
selection = data[0:5]
selection

```{note}
Here we have selected the first five rows (index 0-4) using the integer index.
```

### Selecting several rows and columns

It is also possible to control which columns are chosen when selecting a subset of rows. In this case we will use [pandas.DataFrame.loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) which selects data based on axis labels (row labels and column labels). 

Let's select temperature values (column `depth`) from rows 0-5:

In [ ]:
# Select temp column values on rows 0-5
selection = data.loc[0:5, "depth"]
selection

```{note}
In this case, we get six rows of data (index 0-5)! We are now doing the selection based on axis labels instead of the integer index.
```

It is also possible to select multiple columns when using `loc`. Here, we select the `longitude` and `latitude` columns from a set of rows by passing them inside a list (`.loc[start_index:stop_index, list_of_columns]`):

In [ ]:
# Select columns longitude and latitude on rows 0-5
selection = data.loc[0:5, ["longitude", "latitude"]]
selection

#### Check your understanding

Find the largest earthquake from the latest 20 events in the list.

In [ ]:
data.tail()

In [ ]:
# Here is the solution
data.loc[0:19, "depth"].mean()

### Selecting a single row

You can also select an individual row from a specific position using the `.loc[]` indexing. Here we select all the data values using index 4 (the 5th row):

In [ ]:
# Select one row using index
row = data.loc[4]
row

``.loc[]`` indexing returns the values from that position as a ``pd.Series`` where the indices are actually the column names of those variables. Hence, you can access the value of an individual column by referring to its index using the following format (both should work):


In [ ]:
# Print one attribute from the selected row
row["place"]

### Selecting a single value based on row and column

Sometimes it is enough to access a single value in a DataFrame. In this case, we can use [DataFrame.at](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.at.html#pandas-dataframe-at) instead of `Data.Frame.loc`.

Let's select the temperature (column `TEMP`) on the first row (index `0`) of our DataFrame.

In [ ]:
data.at[0, "place"]

### EXTRA: Selections by integer position

```{admonition} .iloc
`.loc` and `.at` are based on the *axis labels* - the names of columns and rows. Axis labels can be also something else than "traditional" index values. For example, datetime is commonly used as the row index.
`.iloc` is another indexing operator which is based on *integer value* indices. Using `.iloc`, it is possible to refer also to the columns based on their index value. For example,  `data.iloc[0,0]` would return `2019-01-31T04:13:09.980Z` in our example data frame.
    
See the pandas documentation for more information about [indexing and selecting data](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#indexing-and-selecting-data).
```

For example, we could select the first five rows and first five columns based on their index.

In [ ]:
data.iloc[0:5:, 0:5]

To access the value on the first row and second column (`latitude`), the syntax for `iloc` would be:
    

In [ ]:
data.iloc[0, 1]

We can also access individual rows using `iloc`. Let's check out the last row of data:

In [ ]:
data.iloc[-1]

## Filtering and updating data

One really useful feature in pandas is the ability to easily filter and select rows based on a conditional statement.
The following example shows how to select rows when the moment magnitude is greater than 5.0 into variable `mod_eq` ([moderate earthquake](https://www.gns.cri.nz/Home/Learning/Science-Topics/Earthquakes/Monitoring-Earthquakes/Other-earthquake-questions/What-is-the-Richter-Magnitude-Scale)). pandas checks if the condition is `True` or `False` for each row, and returns those rows where the condition is `True`:

In [ ]:
# Check the condition
data["mag"] >= 5.0

In [ ]:
# Select rows with mag 5 and greater
mod_eq = data.loc[data["mag"] >= 5.0]
mod_eq

It is also possible to combine multiple criteria at the same time. Here, we select earthquakes from magnitude 5 that were recorded on the second half of January in 2019 (i.e. `time > 2019-01-15`).
Combining multiple criteria can be done with the `&` operator (AND) or the `|` operator (OR). Notice, that it is often useful to separate the different clauses inside the parentheses `()`.

In [ ]:
# Select rows with mag 5.0 and larger from late January 2019
mod_Eq_latejan = data.loc[(data["mag"] >= 5.0) & (data["time"] >= '2019-01-15')]
mod_Eq_latejan

Now we have a subset of our DataFrame with only rows where the `mag` is 5.0 and greater and the dates in `time` column start from 15th of January.

Notice, that the index values (numbers on the left) are still showing the positions from the original DataFrame. It is possible to **reset** the index using `reset_index()` function that
might be useful in some cases to be able to slice the data in a similar manner as above. By default the `reset_index()` would make a new column called `index` to keep track of the previous
index which might be useful in some cases but not here, so we can omit that by passing parameter `drop=True`.

In [ ]:
# Reset index
mod_eq = mod_eq.reset_index(drop=True)
mod_eq

As can be seen, now the index values goes from 0 to 1.

#### Check your understanding

Find the largest earthquake in the first seven days of January 2019. This time you should select the rows based on a condition for the `time` column!

In [ ]:
# Here's the solution
data["mag"].loc[data["time"] < '2019-01-08'].max()
data


```{admonition} Deep copy
In this lesson, we have stored subsets of a DataFrame as a new variable. In some cases, we are still referring to the original data and any modifications made to the new variable might influence the original DataFrame.
    
If you want to be extra careful to not modify the original DataFrame, then you should take a proper copy of the data before proceeding using the `.copy()` method. You can read more about indexing, selecting data and deep and shallow copies in [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html) and in [this excellent blog post](https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-part-4-c4216f84d388).
```

## Dealing with missing data

Missing values are pretty common in different types of data sets for various reaons (e.g., unrecovered rocks form a drill hole, instrument failure, human error, not enough data).  In this dataset, I have artifically added an entry with a missing value.  This missing values are indicated as `NaN` (not a number). Common procedures to deal with `NaN` values are to either **remove** them from the DataFrame or **fill** them with some value. In pandas both of these options are really easy to do.

Let's first see how we can remove the NoData values (i.e. clean the data) using the [.dropna()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) function. Inside the function you can pass a list of column(s) from which the `NaN` values should found using the `subset` parameter.

In [ ]:
# Drop no data values based on the longitude column
data_clean = data.dropna(subset=['longitude'])
data_clean.head()

As you can see by looking at the table above (and the change in index values), we now have a DataFrame without the NoData values in the `longitude` column.

````{note}
Note that we created an new `data_clean` variable with a version of the dataset where no data are removed. The `.dropna()` function, among other pandas functions can also be applied "in place" which means that the function updates the DataFrame object and returns `None`:
    
```python
data.dropna(subset=['longitude'], inplace=True)
```
````

Another option is to fill the NoData with some value using the `fillna()` function. Here we can fill the missing values in the with value -9999. Note that we are not giving the `subset` parameter this time.

In [ ]:
# Fill na values
filled = data.fillna(-9999.99)
filled.head()

As a result we now have a DataFrame where NoData values are filled with the value -9999.

```{warning}
In many cases filling the data with a specific value is dangerous because you end up modifying the actual data, which might affect the results of your analysis. Hence, use caution when filling missing values. 
    
You might have to fill in no data values for the purposes of saving the data to file in a specific format. For example, some GIS software do not recognize NaN but can instead use a specific value (like -9999) to assign as a "No data value" raster cell. Always pay attention to potential no data values when reading in data files and doing further analysis!
```

## Data type conversions

There are occasions where you'll need to convert data stored within a Series to another data type, for example, from floating point to integer.

Remember, that we already did data type conversions using the [built-in Python functions](https://docs.python.org/3/library/functions.html#built-in-functions) such as `int()` or `str()`.

For values in pandas DataFrames and Series, we can use the `astype()` method.

```{admonition} Truncating versus rounding up
**Be careful with type conversions from floating point values to integers.** The conversion simply drops the stuff to the right of the decimal point, so all values are rounded down to the nearest whole number. For example, 99.99 will be truncated to 99 as an integer, when it should be rounded up to 100.

Chaining the round and type conversion functions solves this issue as the `.round(0).astype(int)` command first rounds the values with zero decimals and then converts those values into integers.
```

In [ ]:
print("Original values:")
data.dropna(subset=["mag"], inplace=True)
data["mag"].head()

In [ ]:
print("Truncated integer values:")
data["mag"].astype(int).head()

In [ ]:
print("Rounded integer values:")
data["mag"].round(0).astype(int).head()

Looks correct now.

## Unique values

Sometimes it is useful to extract the unique values that you have in your column.
We can do that by using `unique()` method.  

In [ ]:
# Get unique mag values
unique = data["mag"].unique()
unique

As a result we get an [array](https://jakevdp.github.io/PythonDataScienceHandbook/02.02-the-basics-of-numpy-arrays.html) of unique values in that column.

```{note}
Sometimes if you have a long list of unique values, you don't necessarily see all the unique values directly as IPython/Jupyter may hide them with an ellipsis `...`. It is, however, possible to see all those values by printing them as a list
```

In [ ]:
# unique values as list
list(unique)


## Sorting data

Quite often it is useful to be able to sort your data (descending/ascending) based on values in some column
This can be easily done with pandas using the `sort_values(by='YourColumnName')` function.

Let's first sort the values on ascending order based on the `TEMP` column:

In [ ]:
# Sort dataframe, ascending
data.sort_values(by="latitude").head(10)

Of course, it is also possible to sort them in descending order with ``ascending=False`` parameter:


In [ ]:
# Sort dataframe, descending
data.sort_values(by="latitude", ascending=False).head(10)

## Writing data to a file

Lastly, it is of course important to be able to write the data that you have analyzed into your computer. This is really handy in pandas as it [supports many different data formats by default](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html).

**The most typical output format by far is a CSV file.** The function `to_csv()` can be used to easily save your data in CSV format. Let's first save the data from our `data` DataFrame into a file called `usgs_eqdata_2019-01_Mw.csv`.

In [ ]:
# define output filename
output_fp = "usgs_eqdata_2019-01_Mw.csv"

# Save dataframe to csv
data.to_csv(output_fp, sep=",")

Now we have the data from our DataFrame saved to a file:

![Text file output1](img/pandas-save-file-1.png)

As you can see, the first value in the datafile now contains the index value of the rows. There are also quite a lot of decimals present in the new columns
that we created. Let's deal with these and save the temperature values from `warm_temps` DataFrame without the index and with only 1 decimal in the floating point numbers.

In [ ]:
# define output filename
output_fp2 = "Phils_moderate-quakes_2019-01.csv"

# Save dataframe to csv
mod_eq.to_csv(output_fp2, sep=",", index=False, float_format="%.1f")

Omitting the index can be done with the `index=False` parameter. Specifying how many decimals should be written can be done with the `float_format` parameter where the text `%.1f` instructs pandas to use 1 decimal in all columns when writing the data to a file (changing the value 1 to 2 would write 2 decimals, etc.)

![Output after float fomatting](img/pandas-save-file-2.png)

As a result you have a "cleaner" output file without the index column, and with only 1 decimal for floating point numbers.

```{admonition} Losing precision
The method above applies to all float type columns. As a consequence, we lose precision in our latitude and longitude, which might not be ideal in other cases. For other uses, you will have to convert the format of a column before you export the dataframe as a .csv file.
```